In [4]:
from fastai.text import *
import html

In [5]:
BOS = 'xbos'  
FLD = 'xfld'  

PATH=Path('data/')

In [6]:
body_df = pd.read_csv(PATH/'train_bodies.csv')

In [7]:
body_df.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [8]:
stances_df = pd.read_csv(PATH/'train_stances.csv')

In [9]:
stances_df.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [10]:
all_df = pd.merge(body_df , stances_df , on='Body ID')

In [27]:
all_df['news'] = all_df['Headline'].str.cat(all_df['articleBody'], sep='ENDh ')

In [28]:
all_df.head()

,Body ID,articleBody,Headline,Stance,news
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated,"Soldier shot, Parliament locked down after gun..."
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated,Tourist dubbed ‘Spider Man’ after spider burro...
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated,Luke Somers 'killed in failed rescue attempt i...
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated,BREAKING: Soldier shot at War Memorial in Otta...
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated,Giant 8ft 9in catfish weighing 19 stone caught...


In [29]:
class_df =pd.DataFrame()

In [30]:
class_df['Stance'] = all_df['Stance']

In [31]:
class_df['news'] = all_df['news']
class_df.head()

,Stance,news
0,unrelated,"Soldier shot, Parliament locked down after gun..."
1,unrelated,Tourist dubbed ‘Spider Man’ after spider burro...
2,unrelated,Luke Somers 'killed in failed rescue attempt i...
3,unrelated,BREAKING: Soldier shot at War Memorial in Otta...
4,unrelated,Giant 8ft 9in catfish weighing 19 stone caught...


In [32]:
class_df.to_pickle('news.pkl')

In [12]:
bodies= body_df['articleBody']

In [15]:
bodies= list(bodies)

In [16]:
headlines = stances_df['Headline']

In [17]:
headlines = list(headlines)

In [18]:
bodies_tok = Tokenizer().proc_all_mp(partition_by_cores(bodies))

/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/bahram/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [19]:
headline_tok = Tokenizer().proc_all_mp(partition_by_cores(headlines))

In [22]:
len(headline_tok) , len(bodies_tok)

(49972, 1683)

In [23]:
trn_bodies,val_bodies = sklearn.model_selection.train_test_split(bodies_tok, test_size=0.1)

In [24]:
trn_headlines,val_headlines = sklearn.model_selection.train_test_split(headline_tok, test_size=0.1)

In [64]:
freq_headline = Counter(p for o in headline_tok for p in o)
freq_headline.most_common(25)

[('t_up', 21695),
 ('in', 13819),
 ("'", 12622),
 (',', 12186),
 ('to', 11722),
 (':', 11673),
 ('-', 9381),
 ('of', 8848),
 ('isis', 7805),
 ('the', 7219),
 (' ', 7053),
 ('a', 6679),
 ('is', 5387),
 ("'s", 4942),
 ('on', 4907),
 ('for', 4800),
 ('at', 3803),
 ('?', 3796),
 ('and', 3598),
 ('by', 3527),
 ('after', 3472),
 ('foley', 3419),
 ('with', 3387),
 ('james', 3147),
 ('video', 3115)]

In [27]:
freq_body = Counter(p for o in bodies_tok for p in o)
freq_body.most_common(25)

[('the', 37719),
 (',', 30230),
 ('.', 28203),
 ('to', 16467),
 ('\n\n', 16113),
 ('a', 15878),
 ('of', 14565),
 ('in', 13123),
 ('and', 12798),
 ('that', 8193),
 ('-', 6446),
 ('is', 6393),
 ('"', 6072),
 ('was', 6063),
 ('on', 6039),
 ('it', 5297),
 ('t_up', 5100),
 ('for', 4981),
 ('he', 4670),
 ('with', 4270),
 ('said', 4222),
 ('as', 3629),
 ('his', 3553),
 ('have', 3534),
 ("'s", 3434)]

In [28]:
max_vocab = 60000
min_freq = 2

In [68]:
itos_headline = [o for o,c in freq_headline.most_common(max_vocab) if c>min_freq]
itos_headline.insert(0, '_pad_')
itos_headline.insert(0, '_unk_')

In [29]:
itos_body = [o for o,c in freq_body.most_common(max_vocab) if c>min_freq]
itos_body.insert(0, '_pad_')
itos_body.insert(0, '_unk_')

In [70]:
stoi_headline = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos_headline)})
len(itos_headline)

3472

In [30]:
stoi_body = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos_body)})
len(itos_body)

11940

In [67]:
trn_lm_headline = np.array([[stoi_headline[o] for o in p] for p in trn_headlines])
val_lm_headline = np.array([[stoi_headline[o] for o in p] for p in val_headlines])

In [31]:
trn_lm_bodies = np.array([[stoi_body[o] for o in p] for p in trn_bodies])
val_lm_bodies = np.array([[stoi_body[o] for o in p] for p in val_bodies])

In [71]:
vs_headline=len(itos_headline)
vs,len(trn_lm_headline)

(11940, 44974)

In [32]:
vs_body=len(itos_body)
vs_body,len(trn_lm_bodies)

(11940, 1514)

In [33]:
em_sz,nh,nl = 400,1150,3

In [34]:
PRE_PATH = PATH/'models'/'wt103'
PRE_LM_PATH = PRE_PATH/'fwd_wt103.h5'

In [35]:
wgts = torch.load(PRE_LM_PATH, map_location=lambda storage, loc: storage)

In [36]:
enc_wgts = to_np(wgts['0.encoder.weight'])
row_m = enc_wgts.mean(0)

In [37]:
itos2 = pickle.load((PRE_PATH/'itos_wt103.pkl').open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

In [38]:
new_w = np.zeros((vs, em_sz), dtype=np.float32)
for i,w in enumerate(itos_body):
    r = stoi2[w]
    new_w[i] = enc_wgts[r] if r>=0 else row_m

In [39]:
wgts['0.encoder.weight'] = T(new_w)
wgts['0.encoder_with_dropout.embed.weight'] = T(np.copy(new_w))
wgts['1.decoder.weight'] = T(np.copy(new_w))

In [40]:
wd=1e-7
bptt=70
bs=64
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [73]:
trn_dl_headline = LanguageModelLoader(np.concatenate(trn_lm_headline), bs, bptt)
val_dl_headline = LanguageModelLoader(np.concatenate(val_lm_headline), bs, bptt)
md_headline= LanguageModelData(PATH, 1, vs, trn_dl_headline, val_dl_headline, bs=bs, bptt=bptt)

In [72]:
trn_dl_bodies = LanguageModelLoader(np.concatenate(trn_lm_bodies), bs, bptt)
val_dl_bodies = LanguageModelLoader(np.concatenate(val_lm_bodies), bs, bptt)
md_body = LanguageModelData(PATH, 1, vs, trn_dl_bodies, val_dl_bodies, bs=bs, bptt=bptt)

In [74]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [76]:
learner_headline= md_headline.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner_headline.metrics = [accuracy]

In [77]:
learner_body= md_body.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner_body.metrics = [accuracy]

In [78]:
learner_headline.freeze_to(-1)

In [79]:
learner_body.freeze_to(-1)

In [80]:
learner_body.model.load_state_dict(wgts)

In [81]:
learner_headline.model.load_state_dict(wgts)

In [82]:
lr=1e-3
lrs = lr

In [85]:
learner_headline.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=20)

epoch      trn_loss   val_loss   accuracy                   
    0      6.424869   6.245771   0.076242  
    1      6.360208   6.150039   0.083199                   
    2      6.243712   6.003581   0.097741                   
    3      6.099594   5.8313     0.114333                   
    4      5.933321   5.630843   0.132824                   
    5      5.747677   5.411896   0.156614                   
    6      5.534624   5.169495   0.18355                    
    7      5.321197   4.920234   0.212949                   
    8      5.095085   4.66418    0.244872                   
    9      4.866994   4.408102   0.275914                   
    10     4.642495   4.164733   0.306313                   
    11     4.462605   3.977893   0.330609                   
    12     4.303864   3.81376    0.353106                   
    13     4.170501   3.676671   0.372231                   
    14     4.068135   3.578912   0.386584                   
    15     3.987982   3.491434   0.398083

[array([3.34052]), 0.41941399986927325]

In [58]:
learner_body.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=5)

epoch      trn_loss   val_loss   accuracy                   
    0      5.150266   4.907632   0.232161  
    1      4.985976   4.742129   0.241877                   
    2      4.797039   4.560748   0.253121                   
    3      4.651877   4.467208   0.258449                   
    4      4.589152   4.434658   0.259685                   



[array([4.43466]), 0.25968457851558924]

In [60]:
learner_body.unfreeze()

In [61]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=15)

epoch      trn_loss   val_loss   accuracy                   
    0      2.194022   3.229676   0.427507  
    1      2.222407   3.221703   0.428337                   
    2      2.150684   3.230138   0.430276                   
    3      2.048796   3.255477   0.433766                   
    4      1.942363   3.289727   0.435256                   
    5      1.907069   3.269384   0.43927                    
    6      1.83231    3.274515   0.442248                   
    7      1.827352   3.23668    0.443653                   
    8      1.756984   3.269422   0.44776                    
 71%|███████   | 106/149 [00:12<00:05,  8.33it/s, loss=1.62]

KeyboardInterrupt: 

In [34]:
learn.save('body_lmbbbnnl')

NameError: name 'learn' is not defined